# Setup

In [32]:
import os
import urllib
import torch

In [33]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False
# For colab
if IN_COLAB:
  import sys
  %git clone https://github.com/elements72/EDiReF-subtask-III.git
  %pip install lightning
  sys.path.append('./EDiReF-subtask-III')
  %cd ./EDiReF-subtask-III
else:
  pass

# Dataset
In this section we will load and pre-process the data

In [34]:
import pandas as pd
from pathlib import Path
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
import numpy as np
from utils import train_model
import torch

In [35]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [36]:
import wandb
wandb.login()

wandb: Currently logged in as: antonio-lopez (ml4cv-handson). Use `wandb login --relogin` to force relogin


True

In [37]:
# Path definitions
colab_path = '/content/drive/MyDrive/NLP/project/'
local_path = './data/'
data_path = Path(local_path)

train_data_path = data_path / 'MELD_train_efr.json'
val_data_path = data_path / 'MELD_val_efr.json'
test_data_path = data_path / 'MELD_test_efr.json'

In [38]:
train_data_raw = pd.read_json(train_data_path)
val_data_raw = pd.read_json(val_data_path)
test_data_raw = pd.read_json(test_data_path)
train_data_raw.head()

,episode,speakers,emotions,utterances,triggers
0,utterance_0,"[Chandler, The Interviewer, Chandler, The Inte...","[neutral, neutral, neutral, neutral, surprise]",[also I was the point person on my company's t...,"[0.0, 0.0, 0.0, 1.0, 0.0]"
1,utterance_1,"[Chandler, The Interviewer, Chandler, The Inte...","[neutral, neutral, neutral, neutral, surprise,...",[also I was the point person on my company's t...,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]"
2,utterance_2,"[Chandler, The Interviewer, Chandler, The Inte...","[neutral, neutral, neutral, neutral, surprise,...",[also I was the point person on my company's t...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ..."
3,utterance_3,"[Chandler, The Interviewer, Chandler, The Inte...","[neutral, neutral, neutral, neutral, surprise,...",[also I was the point person on my company's t...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,utterance_4,"[Joey, Rachel, Joey, Rachel]","[surprise, sadness, surprise, fear]",[But then who? The waitress I went out with la...,"[0.0, 0.0, 1.0, 0.0]"


## Emotion encoding
The emotions are encoded mapping each of them with a specific value. Positive emotion are assigned to positive values the negatives with negatives

In [39]:
emotions = train_data_raw['emotions'].explode().unique()
print(emotions)

['neutral' 'surprise' 'fear' 'sadness' 'joy' 'disgust' 'anger']


In [40]:

# Encode emotions
encoder = LabelEncoder()
encoder.fit(emotions)
print("Mapping:" + str(dict(zip(encoder.classes_, encoder.transform(encoder.classes_)))))

Mapping:{'anger': 0, 'disgust': 1, 'fear': 2, 'joy': 3, 'neutral': 4, 'sadness': 5, 'surprise': 6}


In [41]:
def pre_process(data, test=False):
    tmp = data.copy()
    tmp['emotions'] = data['emotions'].apply(lambda x: encoder.transform(x))
    if not test:
        tmp['triggers'] = data['triggers'].apply(lambda x: [0 if i == np.nan or i == None else i for i in x])
    tmp.drop(columns=['episode'], inplace=True)
    return tmp

In [42]:
train_data = pre_process(train_data_raw)
val_data = pre_process(val_data_raw)
test_data = pre_process(test_data_raw, test=True)

In [43]:
# Show the first row
train_data['triggers'].explode().isna().sum()

0

In [44]:
class UtteranceDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data.index)

    def __getitem__(self, idx):
        return self.data.iloc[idx]

In [45]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [60]:
from baselines import BertBaseline 
import torch

In [47]:
# This collate function takes care of adding padding to the sequences
def collate(batch):
    speakers, emotions, utterances, triggers = zip(*batch)
    emotions = [torch.tensor(e, dtype=torch.long) for e in emotions]
    triggers = [torch.tensor(t, dtype=torch.long) for t in triggers]
    return {
        'speakers': speakers,
        'emotions': torch.nn.utils.rnn.pad_sequence(emotions, batch_first=True, padding_value=-1),
        'utterances': utterances,
        'triggers': torch.nn.utils.rnn.pad_sequence(triggers, batch_first=True, padding_value=-1)
    }

In [ ]:
model = BertBaseline()
train_dataset = UtteranceDataset(train_data)
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=False, num_workers=0, collate_fn=collate)
val_loader = DataLoader(UtteranceDataset(val_data), batch_size=2, shuffle=False, num_workers=0, collate_fn=collate)

In [76]:
lr = 1e-3
epochs = 2

train_model(BertBaseline, 
            "bert_baseline", 
            train_loader,
            val_loader,
            epochs=epochs, 
            hyperparameters={"lr":lr})

Using device: cuda


Seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name                        | Type              | Params
-------------------------------------------------------------------
0  | model                       | BertModel         | 109 M 
1  | emotion_clf                 | CLF               | 99.3 K
2  | trigger_clf                 | CLF               | 98.8 K
3  | f1_train_cumulative_emotion | F1ScoreCumulative | 0     
4  | f1_train_cumulative_trigger | F1ScoreCumulative | 0     
5  | f1_train_dialogues_emotion  | F1ScoreDialogues  | 0     
6  | f1_train_dialogues_trigger  | F1ScoreDialogues  | 0     
7  | f1_val_cumulative_emotion   | F1ScoreCumulative | 0     
8  | f1_val_cumulative_trigger   | F1ScoreCumulative | 0     
9  | f1_val_dialogues_emotion    | F1ScoreDialogues  | 0     
10 | f1_val_dialogues_trigger    | F1Sco

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

AttributeError: 'Tensor' object has no attribute 'compute'